In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import os
import time
import torch
import torch.distributions.constraints as constraints
import pyro
from pyro import poutine
from pyro.optim import Adam
from pyro.infer import SVI, Trace_ELBO, TraceEnum_ELBO, config_enumerate
import pyro.distributions as dist

from sklearn.metrics import matthews_corrcoef as mcc

plt.rcParams.update({"font.size": 16})
pyro.enable_validation(True)

%load_ext autoreload
%autoreload 2

In [4]:
from pyro.infer.mcmc import NUTS, MCMC

In [6]:
true_coefs = torch.tensor([1., 2., 3.])
data = torch.randn(2000, 3)
dim = 3
labels = dist.Bernoulli(logits=(true_coefs * data).sum(-1)).sample()

def model(data):
    coefs_mean = torch.zeros(dim)
    coefs = pyro.sample('beta', dist.Normal(coefs_mean, torch.ones(3)))
    y = pyro.sample('y', dist.Bernoulli(logits=(coefs * data).sum(-1)), obs=labels)
    return y

nuts_kernel = NUTS(model, adapt_step_size=True)
mcmc_run = MCMC(nuts_kernel, num_samples=500, warmup_steps=300).run(data)
posterior = mcmc_run.marginal('beta').empirical['beta']
posterior.mean  # doctest: +SKIP

Sample: 100%|██████████| 800/800 [00:07<00:00, 120.05it/s, step size=7.01e-01, acc. rate=0.920]


tensor([0.8947, 1.7431, 2.7837])

In [13]:
posterior.sample()

tensor([0.9807, 1.9144, 2.8883])